In [1]:
import numpy as np
import os
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import string

In [2]:
files = []
fileMap = {}
x = 0
for filename in os.listdir('./Files'):
    try:
        f = open("./Files/"+filename, "r")
        files.append(f.read())
        fileMap[x] = filename
        x += 1
    except:
        f = open("./Files/"+filename, "rb")
        files.append(f.read().decode('utf-8', 'backslashreplace'))
        fileMap[x] = filename
        x += 1
        
print("Files read: ", x)

Files read:  1133


# Question 1:

## a) Preprocessing:

In [3]:
lemmatizer = WordNetLemmatizer() 
stopWords = set(stopwords.words('english'))

In [4]:
def preProcess(file):
    # Keep only alphabets and numbers
    file = re.sub(r'[^a-zA-Z0-9]+', ' ', file)
    # Convert to lower case:
    file = file.lower()
    # Stopword removal
    file = [word for word in file.split() if word not in stopWords]
    # Lemmatize
    file = [lemmatizer.lemmatize(word) for word in file]    #lemmatize the words
    return file

In [5]:
splitFiles = []
for file in files:
    splitFiles.append(set(preProcess(file)))

## b) Unigram Inverted Index:

In [6]:
invertedIndex = {}
for idx, file in enumerate(splitFiles): # For each file. numbered 0-n
    for word in file: # for each unique word in that file.
        if word not in invertedIndex: # insert into dictionary the document index.
            invertedIndex[word] = []
        invertedIndex[word].append(idx)

## c) Boolean Functions:

In [7]:
def getList(x, index):
    try:
        xList = index[x]
        return xList
    except:
        return []

#### 1. x OR y:

In [8]:
def xORy(xList, yList):
    xLen = len(xList)
    yLen = len(yList)
    
    # if one is empty return the other.
    if xLen == 0:
        return (yList, 1)
    if yLen == 0:
        return (xList, 1)
    
    # merge both
    comparisons = 0
    a = 0
    b = 0
    ans = set()
    while a < xLen and b < yLen:
        # Compare values
        if xList[a] <= yList[b]:
            ans.add(xList[a])
            a += 1
        else:
            ans.add(yList[b])
            b += 1
        
        comparisons += 1
            
    # if some values remaining from xList then add them.
    while a < xLen:
        ans.add(xList[a])
        a += 1

    # if some values remaining from yList then add them.
    while b < yLen:
        ans.add(yList[b])
        b += 1
        
    return (list(ans), comparisons)

#### 2. x AND y:

In [9]:
def xANDy(xList, yList):
    xLen = len(xList)
    yLen = len(yList)
    
    # if one is empty answer is empty.
    if xLen == 0:
        return (xList, 1)
    if yLen == 0:
        return (yList, 1)
    
    # merge both
    comparisons = 0
    a = 0
    b = 0
    ans = set()
    while a < xLen and b < yLen:
        # Compare values
        if xList[a] < yList[b]:
            a += 1
        elif xList[a] > yList[b]:
            b += 1
        else:
            ans.add(yList[b])
            a += 1
            b += 1
        
        comparisons += 1
        
    return (list(ans), comparisons)

#### 3. x AND NOT y:

In [10]:
def xANDNOTy(xList, yList):
    xLen = len(xList)
    yLen = len(yList)
    
    # if x is empty return empty
    if xLen == 0:
        return ([], 1)
    # if y is empty return x
    if yLen == 0:
        return (xList, 1)
    
    # merge both
    comparisons = 0
    a = 0
    b = 0
    ans = set()
    while a < xLen and b < yLen:
        # Compare values
        if xList[a] < yList[b]:
            ans.add(xList[a])
            a += 1
        elif xList[a] > yList[b]:
            b += 1
        else:
            a += 1
            b += 1
        
        comparisons += 1
        
    return (list(ans), comparisons)

#### 4. x OR NOT y:

In [11]:
def xORNOTy(xList, yList):
    xLen = len(xList)
    yLen = len(yList)
    
    ans = set(range(len(files)))
    comparisons = 0
    for yW in yList:
        if yW not in xList: # remove everyword in yList and not in xList
            ans.remove(yW)
        comparisons += 1
    
    return (list(ans), comparisons)

## d) Queries:

In [15]:
def takeInput():
    query = input("Enter string without operators: ")
    op = input("Enter operators (comma seperated): ")
    op = op.split(",")
    op = [x.strip() for x in op]
    query = preProcess(query)
    print(query, "\n", op)
    return(query, op)

In [44]:
def getResults(queryString, operators):
    if len(queryString) == 0:
        print("Total number of documents: ", 0)
        print("Total number of comparisons: ", 0)
        return
    
    xList = getList(queryString[0], invertedIndex)
    opIdx = 0
    nextListIdx = 1
    
    # Read query left to right and get results
    totalComps = 0
    while nextListIdx < len(queryString) and opIdx < len(operators):
        yList = getList(queryString[nextListIdx], invertedIndex)
        
        if operators[opIdx] == 'OR':
            res = xORy(xList, yList)
            xList = res[0]
            totalComps += res[1]
        elif operators[opIdx] == 'AND':
            res = xANDy(xList, yList)
            xList = res[0]
            totalComps += res[1]
        elif operators[opIdx] == 'AND NOT':
            res = xANDNOTy(xList, yList)
            xList = res[0]
            totalComps += res[1]
        elif operators[opIdx] == 'OR NOT':
            res = xORNOTy(xList, yList)
            xList = res[0]
            totalComps += res[1]
        else:
            print("Wrong operator found")
            break
        
        opIdx += 1
        nextListIdx += 1
    
    print("Total documents: ", len(xList))
    print("Total comparisons: ", totalComps)
    return xList

In [24]:
(q, op) = takeInput()

Enter string without operators: lion stood thoughtfully for the moment
Enter operators (comma seperated): OR,OR, OR
['lion', 'stood', 'thoughtfully', 'moment'] 
 ['OR', 'OR', 'OR']


In [39]:
fileIdx = getResults(q, op)

Total documents:  211
Total comparisons:  353


In [40]:
for f in fileIdx:
    print(f, fileMap[f])

1028 commutin.jok
1031 grail.txt
523 suicide2.txt
524 cartoon.laws
531 radiolaf.hum
534 hackingcracking.txt
535 snapple.rum
22 kanalx.txt
25 bmdn01.txt
540 myheart.hum
32 tnd.1
33 annoy.fascist
544 dingding.hum
1056 luggage.hum
1057 facedeth.txt
38 stone.hum
558 flux_fix.txt
560 hackmorality.txt
50 jayjay.txt
562 badday.hum
565 luvstory.txt
567 mash.hum
569 dthought.txt
1081 homebrew.txt
571 idr2.txt
1087 conan.txt
1089 lion.jok
1091 popmusi.hum
1092 montpyth.hum
70 insults1.txt
73 reasons.txt
76 gd_gal.txt
589 golnar.txt
1100 phorse.hum
80 incarhel.hum
592 tfepisod.hum
595 econridl.fun
597 nigel10.txt
86 sfmovie.txt
1110 soleleer.hum
602 news.hum
604 nihgel_8.9
94 bw-phwan.hat
95 stuf11.txt
96 sw_err.txt
97 pizzawho.hum
606 prac3.jok
608 prac2.jok
1120 dead3.txt
101 butwrong.hum
1121 fascist.txt
615 gd_ql.txt
105 calculus.txt
617 christop.int
107 iremember
108 stuf10.txt
621 computer.txt
110 oliver02.txt
1131 xibovac.txt
113 top10st2.txt
625 is_story.txt
118 m0dzmen.hum
631 clancy.txt

In [46]:
(q, op) = takeInput()
x = getResults(q, op)

Enter string without operators: telephone,paved, roads
Enter operators (comma seperated): OR NOT, AND NOT
['telephone', 'paved', 'road'] 
 ['OR NOT', 'AND NOT']
Total documents:  996
Total comparisons:  1137


In [47]:
for f in x:
    print(f, fileMap[f])

0 naivewiz.hum
1 lawsuniv.hum
2 potty.txt
3 adrian_e.faq
4 pecker.txt
5 miranda.hum
6 imbecile.txt
7 fish.rec
8 btscke04.des
9 poli_t.ics
10 cucumber.jok
11 lp-assoc.txt
12 brownie.rec
13 egglentl.vgn
14 chainltr.txt
15 murph.jok
16 farsi.txt
17 goforth.hum
18 socks.drx
19 fed.txt
20 htswfren.txt
22 kanalx.txt
23 wood
24 curse.txt
25 bmdn01.txt
26 vegan.rcp
27 pbcookie.des
28 prover_w.iso
29 1st_aid.txt
30 law.sch
31 merry.txt
32 tnd.1
33 annoy.fascist
34 spoonlis.txt
36 top10st1.txt
37 takenote.jok
38 stone.hum
39 planetzero.txt
40 transp.txt
41 newmex.hum
42 prayer.hum
43 middle.age
44 nukwaste
45 phxbbs-m.txt
46 avengers.lis
47 woolly_m.amm
48 blkbean.txt
50 jayjay.txt
51 test.hum
52 gack!.txt
53 nukewar.jok
54 eandb.drx
55 margos.txt
56 btscke05.des
57 cgs_lst.txt
59 jimhood.txt
60 cereal.txt
61 oasis
62 hell.txt
63 fwksfun.hum
64 cheapin.la
65 corporat.txt
66 planeget.hum
67 jokeju07.txt
68 btcisfre.hum
69 dead-r
71 pure.mat
72 signatur.jok
73 reasons.txt
74 shrink.news
75 smurf_c

# Question 2:

## a) Preprocess:

In [ ]:
def preProcess2(file):
    # Convert file to lowercase.
    file = file.lower()
    
    # Tokenize and remove stopwords.
    file = " ".join([word for word in file.split() if word not in stopWords])
    
    # word tokenize.
    file = word_tokenize(file)
    
    # Remove stopwords remaining
    file = [word for word in file if word not in stopWords]
    
    # Remove punctuations
    file = [x.translate(str.maketrans('', '', string.punctuation)) for x in file]
    
    # Remove len <= 2 tokens and lemmatize the words.
    newFile = []
    for word in file:
        word = word.strip()
        if len(word) > 2:
            newFile.append(lemmatizer.lemmatize(word))
    return newFile

## b) Positional Index:

In [ ]:
positionalIndex = {}
for idx, file in enumerate(files):
    file = preProcess2(file)
    for wordIdx, word in enumerate(file):
        # Word not existed before.
        if word not in positionalIndex:
            positionalIndex[word] = {}
            positionalIndex[word]["docList"] = [idx]
            
        # Word existed, but occuring first time in this document.
        if idx not in positionalIndex[word]:
            positionalIndex[word][idx] = []
            
        # Insert the word location for this document.
        positionalIndex[word][idx].append(wordIdx)
        if positionalIndex[word]["docList"][-1] != idx:
            positionalIndex[word]["docList"].append(idx)

## c) Phrase Query Function:

##### Phrase query support for <= 5

In [ ]:
def getPositionalList(word):
    if word not in positionalIndex:
        return {
            "docList": []
        }
    return positionalIndex[word]

In [ ]:
def takePhraseInput():
    q = input("Enter query: ")
    q = preProcess2(q)
    print(q)
    return q

In [ ]:
def getPhraseQueryResult(query):
    if len(query) == 0:
        return (0, [])
    
    # get query for a single word
    xList = getPositionalList(query[0])
    
    # initialize answer.
    ansList = xList["docList"]
    
    # for next words.
    for word in query[1:]:
        yList = getPositionalList(word)
        # get common documents.
        commonDocs = xANDy(xList["docList"], yList["docList"])[0]
        newList = []
        
        # in common documents check if words are adjacent.
        for docId in commonDocs:
            # get positions of the words in this docId
            xPos = xList[docId]
            yPos = yList[docId]
            
            a = 0
            b = 0
            
            while a < len(xPos) and b < len(yPos):
                if xPos[a] == yPos[b]-1:
                    newList.append(docId)
                    a += 1
                    b += 1
                    break
                elif xPos[a] < yPos[b]:
                    a += 1
                else:
                    b += 1
            
        xList = yList
        
        ansList = xANDy(ansList, newList)[0]
    
    return (len(ansList), ansList)

In [ ]:
q = takePhraseInput()

In [ ]:
res = getPhraseQueryResult(q)
print(res)

In [ ]:
for filename in res[1]:
    print(fileMap[filename])

In [ ]:
q = takePhraseInput()

In [ ]:
res = getPhraseQueryResult(q)
print(res)

In [ ]:
for filename in res[1]:
    print(fileMap[filename])

In [ ]:
q = takePhraseInput()
res = getPhraseQueryResult(q)
print(res)
for filename in res[1]:
    print(fileMap[filename])